In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import utilities
import sys 
import json
import importlib
sys.path.append('../')
reset = importlib.import_module("utilities.reset")
reset_pipeline = reset.reset_pipeline

# load secrets from a .env file using python-dotenv
from dotenv import load_dotenv
import os
load_dotenv("../../.env")
MY_API_KEY = os.getenv('MY_API_KEY')
MY_API_URL = os.getenv('MY_API_URL')

# import krixik and initialize it with your personal secrets
from krixik import krixik
krixik.init(api_key = MY_API_KEY, 
            api_url = MY_API_URL)

SUCCESS: You are now authenticated.


In [14]:
from utilities.converter import convert_all_notebooks
from utilities.converter import collect_mkdocks_toc
from utilities.link_check import check_file_links
from utilities import base_dir

In [15]:
base_dir

'/Users/jeremywatt/Desktop/krixik/code/krixik-docs'

In [4]:
convert_all_notebooks()

In [5]:
all_docs = collect_mkdocks_toc()

In [39]:
import re

def get_code_from_markdown(lines: list[str], *, language: str = "python") -> list[str]:
    """Outputs extracted code blocks from a list of strings of markdown text"""
    # from: https://github.com/tassaron/get_code_from_markdown
    regex = re.compile(
        r"(?P<start>^```(?P<block_language>(\w|-)+)\n)(?P<code>.*?\n)(?P<end>```)",
        re.DOTALL | re.MULTILINE,
    )
    blocks = [
        (match.group("block_language"), match.group("code"))
        for match in regex.finditer("".join(lines))
    ]
    return [block for block_language, block in blocks if block_language == language]


def gather_pipeline_names(md_filepath: str) -> list:
    try:
        with open(md_filepath, 'r', encoding='utf-8') as file:
            markdown_content = file.read()
        code_blocks = get_code_from_markdown([markdown_content])
        create_pattern = r'\.create_pipeline\([^)]*\)'
        name_pattern = r'name="([^"]*)"'
        pipeline_names = []
        for block in code_blocks:
            create_matches = []
            create_matches = re.findall(create_pattern, block)
            if len(create_matches) > 0:
                name_matches = []
                for match in create_matches:
                    name_matches = re.findall(name_pattern, match)
                    for item in name_matches:
                        pipeline_names.append(item)
        if len(pipeline_names) > 0:
            pipeline_names = list(set(pipeline_names))
        return pipeline_names
    except Exception as e:
        raise Exception(f"FAILURE: gather_pipeline_names failed on file {markdown_file} with exception {e}")

def duplicate_name_check():
    try:
        # collect all paths to md docs
        all_docs = collect_mkdocks_toc()

        # collect all pipeline names
        all_names = []
        all_md_names = []
        for i in range(len(all_docs)):
            md_filepath = base_dir + "/docs/" +  all_docs[i]
            pipeline_names = gather_pipeline_names(md_filepath)
            all_md_names.append(pipeline_names)
            for name in pipeline_names:
                all_names.append(name)
                
        # determine duplicates
        my_dict = {i:all_names.count(i) for i in all_names}
        keys = list(my_dict.keys())
        values = list(my_dict.values())
        duplicates = [key for key in keys if my_dict[key] > 1]

        if len(duplicates) > 0:
            # gather all paths to mds containing duplicate pipeline names
            duplicates_dict = {v:[] for v in duplicates}
            
            for d in duplicates:
                for i in range(len(all_docs)):
                    docs_names = all_md_names[i]
                    if d in docs_names:
                        duplicates_dict[d].append(all_docs[i])
                        
            print(f"FAILURE: the following pipeline names are found in multiple markdown docs: {duplicate_dicts}")
            return False
        return True
    except Exception as e:
        print(f"FAILURE: duplicate_name_check failed with exception {e}")
        return False

In [49]:
dupcliates

[]